# HW 4

In [1]:
import pickle
import pandas as pd

In [2]:
with open('model.bin', 'rb') as f_in:
    dv, lr = pickle.load(f_in)

In [3]:
categorical = ['PUlocationID', 'DOlocationID']


def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [4]:
year = 2021
month = 2

df = read_data(f'https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_{year:04d}-{month:02d}.parquet')

In [5]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = lr.predict(X_val)

## Q1. Notebook

In [6]:
import numpy as np

In [7]:
np.mean(y_pred)

16.191691679979066

## Q2. Preparing the output

In [8]:
output_file = 'predictions'
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')
df['pred'] = y_pred

In [9]:
df_result = df.loc[:, ['ride_id', 'pred']]

In [10]:
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [11]:
from pathlib import Path
file_size = Path('predictions').stat().st_size
print(f'Size of the file is {round(file_size/1024**2, 2)} MB')

Size of the file is 18.8 MB


## Q3. Creating the scoring script

jupyter nbconvert "starter.ipynb" --to script --output ride_duration

## Q4. Virtual environment

"sha256:08ef968f6b72033c16c479c966bf37ccd49b06ea91b765e1cc27afefe723920b"

## Q5.Parameterize the script 

In [12]:
%run predict.py 2021 03

16.298821614015107


## Q6. Docker container

In [13]:
%run predict.py 2021 04

16.551162930515858
